# Bayesian Inference on Medical Trial Data

This notebook applies Bayesian inference to medical trial data. The goal is to estimate the probability of patient improvement in treatment and control groups using both sampling and function-fitting approaches.

## Data Overview

We have summarized trial data for two groups:

| Group     | Improved | Total Patients |
|-----------|---------:|---------------|
| Treatment | 107      | 141           |
| Control   | 57       | 121           |

We aim to estimate the probability of improvement for each group using Bayesian methods.

In [ ]:
# Libraries
import pymc as pm
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

# Dataset
trial_data = {
    'treatment': {'improved': 107, 'patients': 141},
    'control': {'improved': 57, 'patients': 121}
}

## Modeling the Control Group

The number of improved patients is modeled as a Binomial distribution. Since we have no strong prior beliefs, we use a uniform prior (Beta(1,1)) for the probability of improvement.

**Model:**
- Likelihood: x ~ Binomial(n, p)
- Prior: p ~ Beta(1,1)
- Posterior: proportional to Likelihood × Prior

In [ ]:
# Bayesian model for Control group
with pm.Model() as control_model:
    p_control = pm.Beta('p_control', alpha=1, beta=1)
    x_control = pm.Binomial('x_control', 
                             n=trial_data['control']['patients'], 
                             p=p_control, 
                             observed=trial_data['control']['improved'])

### Posterior Sampling
We sample from the posterior using PyMC's NUTS sampler to estimate the probability distribution of improvement.

In [ ]:
with control_model:
    control_trace = pm.sample()

posterior_samples = control_trace.posterior.p_control.values.flatten()

# Histogram of posterior samples
plt.figure(figsize=(8,4))
plt.hist(posterior_samples, bins=20, density=True, edgecolor='white', label='Sampling Approximation')

# Exact posterior for comparison
x = np.linspace(0,1,500)
y = sts.beta.pdf(x, trial_data['control']['improved']+1, 
                    trial_data['control']['patients'] - trial_data['control']['improved'] +1)
plt.plot(x, y, label='Analytical Posterior')
plt.xlabel('Probability of Improvement')
plt.ylabel('Density')
plt.title('Posterior Distribution for Control Group')
plt.legend()
plt.show()

**Interpretation:**
The posterior is centered around ~0.47, indicating the control group has roughly a 47% probability of improvement. Sampling aligns well with the analytical Beta posterior, confirming the model.

## Posterior Approximation via Function Fitting
To approximate the posterior faster, we can use variational inference (ADVI).

In [ ]:
with control_model:
    approx = pm.fit()
    fit_samples = approx.sample(4000)

posterior_fit = fit_samples.posterior.p_control.values.flatten()

plt.figure(figsize=(8,4))
plt.hist(posterior_fit, bins=20, density=True, edgecolor='white', label='Function Fitting Approximation')
plt.plot(x, y, label='Analytical Posterior')
plt.xlabel('Probability of Improvement')
plt.ylabel('Density')
plt.title('Posterior Approximation for Control Group (ADVI)')
plt.legend()
plt.show()

**Reflection:**
- Sampling is more precise but slower.
- Function fitting (ADVI) is faster and provides a reasonable approximation.
- Both methods indicate the control group's improvement probability is lower than the treatment group.

## Treatment Group Analysis
We repeat the Bayesian analysis for the treatment group.

In [ ]:
# Bayesian model for Treatment group
with pm.Model() as treatment_model:
    p_treatment = pm.Beta('p_treatment', alpha=1, beta=1)
    x_treatment = pm.Binomial('x_treatment', 
                              n=trial_data['treatment']['patients'], 
                              p=p_treatment, 
                              observed=trial_data['treatment']['improved'])

In [ ]:
with treatment_model:
    treatment_trace = pm.sample()

posterior_samples_treatment = treatment_trace.posterior.p_treatment.values.flatten()

plt.figure(figsize=(8,4))
plt.hist(posterior_samples_treatment, bins=20, density=True, edgecolor='white', label='Sampling Approximation')

y_treat = sts.beta.pdf(x, trial_data['treatment']['improved']+1, 
                          trial_data['treatment']['patients'] - trial_data['treatment']['improved'] +1)
plt.plot(x, y_treat, label='Analytical Posterior')
plt.xlabel('Probability of Improvement')
plt.ylabel('Density')
plt.title('Posterior Distribution for Treatment Group')
plt.legend()
plt.show()

**Interpretation:**
The treatment group posterior centers around ~0.76, indicating treatment significantly improves patient outcomes.

## Comparing Treatment vs Control
Visualizing the posterior distributions together illustrates the treatment's effect.

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(posterior_samples, bins=20, density=True, alpha=0.5, label='Control')
plt.hist(posterior_samples_treatment, bins=20, density=True, alpha=0.5, label='Treatment')
plt.xlabel('Probability of Improvement')
plt.ylabel('Density')
plt.title('Comparison: Treatment vs Control')
plt.legend()
plt.show()

## Conclusion

- Bayesian inference confirms the treatment group has a higher probability of improvement (~0.76) compared to the control (~0.47).
- Both sampling and function-fitting methods give consistent insights.
- This demonstrates the power of Bayesian methods for quantifying uncertainty and comparing interventions.